<a href="https://colab.research.google.com/github/strg-frontier/pytorch_wine_quality/blob/master/resnet_%E8%BB%A2%E7%A7%BB%E5%AD%A6%E7%BF%92%E6%AF%94%E8%BC%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision import transforms

from tqdm import tqdm

In [3]:
from google.colab import files
uploaded = files.upload()

Saving taco_and_burrito.tar.gz to taco_and_burrito.tar.gz


In [0]:
!tar -xzf taco_and_burrito.tar.gz

In [8]:
!ls taco_and_burrito

test  train


In [0]:
base_dir = './taco_and_burrito'
train_imgs = ImageFolder(base_dir + '/train/',
                         transform=transforms.Compose([
                             transforms.RandomCrop(224),
                             transforms.ToTensor()]
                         ))
test_imgs = ImageFolder(base_dir + '/test/',
                         transform=transforms.Compose([
                             transforms.RandomCrop(224),
                             transforms.ToTensor()]
                         ))

train_loader = DataLoader(
    train_imgs, batch_size=32, shuffle=True)
test_loader = DataLoader(
    test_imgs, batch_size=32, shuffle=False)

In [10]:
train_imgs.classes

['burrito', 'taco']

In [11]:
resnet = models.resnet18(pretrained=True)
resnet

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
for p in resnet.parameters():
    p.requires_grad=False

fc_input_dim = resnet.fc.in_features
resnet.fc = nn.Linear(fc_input_dim, 2)
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [0]:
# 最適化手法のパラメータ設定
optimizer = optim.SGD(resnet.fc.parameters(), lr=0.1, momentum=0.9)
# loss関数の定義
criterion = nn.CrossEntropyLoss()


In [15]:
import datetime

def train(epoch):
    total_loss = 0
    total_size = 0
    resnet.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = resnet(data)
        loss = criterion(output, target)
        total_loss += loss.item()
        total_size += data.size(0)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            now = datetime.datetime.now()
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                now,
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), total_loss / total_size))

for epoch in range(1, 10 + 1):
    train(epoch)

[2020-02-16 10:47:23.297740] Train Epoch: 1 [0/712 (0%)]	Average loss: 0.023112
[2020-02-16 10:48:14.330394] Train Epoch: 2 [0/712 (0%)]	Average loss: 0.242474
[2020-02-16 10:49:04.886860] Train Epoch: 3 [0/712 (0%)]	Average loss: 0.150537
[2020-02-16 10:49:55.317779] Train Epoch: 4 [0/712 (0%)]	Average loss: 0.098968
[2020-02-16 10:50:45.393990] Train Epoch: 5 [0/712 (0%)]	Average loss: 0.105158
[2020-02-16 10:51:35.291654] Train Epoch: 6 [0/712 (0%)]	Average loss: 0.226107
[2020-02-16 10:52:25.867327] Train Epoch: 7 [0/712 (0%)]	Average loss: 0.127812
[2020-02-16 10:53:16.880704] Train Epoch: 8 [0/712 (0%)]	Average loss: 0.030538
[2020-02-16 10:54:06.833161] Train Epoch: 9 [0/712 (0%)]	Average loss: 0.064502
[2020-02-16 10:54:56.920403] Train Epoch: 10 [0/712 (0%)]	Average loss: 0.189144


In [0]:
resnet = resnet.eval()
pred = []
Y = []
for i, (x,y) in enumerate(test_loader):
    with torch.no_grad():
        output = resnet(x)
    pred += [int(l.argmax()) for l in output]
    Y += [int(l) for l in y]

In [17]:
correct=0
for i in range(len(Y)):
    correct += Y[i]==pred[i]
print(correct/len(Y))

0.75


In [24]:
resnet.layer4[1]

BasicBlock(
  (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [0]:
resnet2 = models.resnet18(pretrained=True)
for p in resnet2.parameters():
    p.requires_grad=False

fc_input_dim = resnet2.fc.in_features
resnet2.fc = nn.Linear(fc_input_dim, 2)

resnet2.layer4[1].conv1 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
resnet2.layer4[1].conv2 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [69]:
resnet2.layer4[1]

BasicBlock(
  (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [70]:
# 最適化手法のパラメータ設定
optimizer = optim.SGD(resnet2.parameters(), lr=0.1, momentum=0.9)
# loss関数の定義
criterion = nn.CrossEntropyLoss()

def train2(epoch):
    total_loss = 0
    total_size = 0
    resnet.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = resnet2(data)
        loss = criterion(output, target)
        total_loss += loss.item()
        total_size += data.size(0)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            now = datetime.datetime.now()
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                now,
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), total_loss / total_size))

for epoch in range(1, 10 + 1):
    train2(epoch)

[2020-02-16 12:01:03.021680] Train Epoch: 1 [0/712 (0%)]	Average loss: 0.022324
[2020-02-16 12:02:00.322494] Train Epoch: 2 [0/712 (0%)]	Average loss: 0.063023
[2020-02-16 12:02:57.540327] Train Epoch: 3 [0/712 (0%)]	Average loss: 0.092174
[2020-02-16 12:03:54.932193] Train Epoch: 4 [0/712 (0%)]	Average loss: 0.005381
[2020-02-16 12:04:52.259216] Train Epoch: 5 [0/712 (0%)]	Average loss: 0.003892
[2020-02-16 12:05:49.661782] Train Epoch: 6 [0/712 (0%)]	Average loss: 0.009463
[2020-02-16 12:06:47.022738] Train Epoch: 7 [0/712 (0%)]	Average loss: 0.001427
[2020-02-16 12:07:44.218849] Train Epoch: 8 [0/712 (0%)]	Average loss: 0.001250
[2020-02-16 12:08:41.876961] Train Epoch: 9 [0/712 (0%)]	Average loss: 0.012521
[2020-02-16 12:09:38.561927] Train Epoch: 10 [0/712 (0%)]	Average loss: 0.029689


In [71]:
resnet2 = resnet2.eval()
pred2 = []
Y2 = []
for i, (x,y) in enumerate(test_loader):
    with torch.no_grad():
        output = resnet2(x)
    pred2 += [int(l.argmax()) for l in output]
    Y2 += [int(l) for l in y]
correct2=0
for i in range(len(Y2)):
    correct2 += Y2[i]==pred2[i]
print(correct2/len(Y2))

0.8333333333333334
